# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
import glob

from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata

import warnings 
warnings.filterwarnings('ignore') 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [3]:
df_full = pd.read_parquet('../BIS_data/Final_BIS_Data.parquet')
df_full = df_full.reset_index(drop=True)
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2138275 entries, 0 to 2138274
Data columns (total 6 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   pca_0                   float64
 1   pca_1                   float64
 2   pca_2                   float64
 3   pca_3                   float64
 4   pca_4                   float64
 5   laundering_schema_type  int64  
dtypes: float64(5), int64(1)
memory usage: 97.9 MB


# 2. Split Data

In [4]:
# ---------------------------
# 2. Split Data into Train and Test Sets BEFORE Undersampling
# ---------------------------
X = df_full.drop('laundering_schema_type', axis=1)
y = df_full['laundering_schema_type']

# Use stratify=y to preserve the class distribution in the test set.
X_train_ori, X_test_ori, y_train_ori, y_test_ori = train_test_split(
    X, y, test_size=0.1, random_state=42, stratify=y
)

# 3. SMOTE

### 3.1 Undersampling + SMOTE

In [6]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from collections import Counter

# =============================================================================
# 2. Define thresholds for undersampling and oversampling.
# For production, set these to 1_000_000. Here we use 500 for a demonstration.
# =============================================================================
majority_class = 0
minority_class = 1

undersample_target = 1000000  # Replace with 1_000_000 in production
oversample_target  = 1000000  # Replace with 1_000_000 in production

# =============================================================================
# 3. Separate features and target.
# =============================================================================

# =============================================================================
# 4. Undersample the majority class.
# This step reduces the majority class to the specified threshold.
# =============================================================================
rus = RandomUnderSampler(sampling_strategy={majority_class: undersample_target}, random_state=42)

X_under, y_under = rus.fit_resample(X_train_ori.values, y_train_ori.values)

print("After undersampling:", Counter(y_under))
# At this point, the majority class count is now `undersample_target` while the minority remains unchanged.

# =============================================================================
# 5. Oversample the minority class using SMOTE.
# This step increases the minority class to the specified threshold.
# =============================================================================
# SMOTE requires at least (k_neighbors + 1) samples in the minority class. Adjust if needed.
current_minority_count = Counter(y_under)[minority_class]

k_neighbors = 5 if current_minority_count > 5 else current_minority_count - 1

smote = SMOTE(sampling_strategy={minority_class: oversample_target}, k_neighbors=k_neighbors, random_state=42)

X_res, y_res = smote.fit_resample(X_under, y_under)
print("After oversampling:", Counter(y_res))
# Now, both classes should have the target number of samples.

# =============================================================================
# 6. Combine the resampled data into a DataFrame.
# =============================================================================
df_resampled = pd.DataFrame(X_res, columns=[f'pca_{i}' for i in range(X_res.shape[1])])
df_resampled['laundering_schema_type'] = y_res

print("\nResampled dataset shape:", df_resampled.shape)
print("Resampled class distribution:")
print(df_resampled['laundering_schema_type'].value_counts())
df_resampled = df_resampled.sample(frac=1)

After undersampling: Counter({np.int64(0): 1000000, np.int64(1): 110620})
After oversampling: Counter({np.int64(0): 1000000, np.int64(1): 1000000})

Resampled dataset shape: (2000000, 6)
Resampled class distribution:
laundering_schema_type
0    1000000
1    1000000
Name: count, dtype: int64


In [7]:
# Separate features and labels for the undersampled training data.
X_train = df_resampled.drop('laundering_schema_type', axis=1).values
y_train = df_resampled['laundering_schema_type'].values
X_train = X_train.astype(np.float32)

# For the test set, keep the original (imbalanced) distribution.
X_test = X_test_ori.values
y_test = y_test_ori.values
X_test = X_test.astype(np.float32)

# Convert the datasets to PyTorch tensors.
X_train = torch.tensor(X_train, dtype=torch.float).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_test  = torch.tensor(X_test, dtype=torch.float).to(device)
y_test  = torch.tensor(y_test, dtype=torch.long).to(device)

### 3.2 Default SMOTE

In [5]:
# ---------------------------
# 3. SMOTE Only the Training Set
# ---------------------------
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_ori, y_train_ori)

In [6]:
# Separate features and labels for the undersampled training data.
X_train = X_train_smote.values
y_train = y_train_smote.values
X_train = X_train.astype(np.float32)

# For the test set, keep the original (imbalanced) distribution.
X_test = X_test_ori.values
y_test = y_test_ori.values
X_test = X_test.astype(np.float32)

# Convert the datasets to PyTorch tensors.
X_train = torch.tensor(X_train, dtype=torch.float).to(device)
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
X_test  = torch.tensor(X_test, dtype=torch.float).to(device)
y_test  = torch.tensor(y_test, dtype=torch.long).to(device)


# 4. Define Model

In [7]:
# ---------------------------
# 4. Define the MLP Model
# ---------------------------
class ResidualBlock(nn.Module):
    def __init__(self, hidden_dim):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
        )
        self.relu = nn.ReLU()
        
    def forward(self, x):
        residual = x
        out = self.block(x)
        out += residual  # Skip connection
        out = self.relu(out)
        return out

class FraudResNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_blocks=3):
        super(FraudResNet, self).__init__()
        # Initial projection layer
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )
        
        # Residual blocks
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(hidden_dim) for _ in range(num_blocks)]
        )
        
        # Final classification layer
        self.output_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.res_blocks(x)
        x = self.output_layer(x)
        return x

In [8]:
input_dim = X_train.shape[1]
hidden_dim = 32
output_dim = 2  # Two neurons for two classes
num_blocks = 3  # Number of residual blocks

model = FraudResNet(input_dim, hidden_dim, output_dim, num_blocks).to(device)
# model = FraudResNetCNN(input_dim, num_channels=64, output_dim=2, num_blocks=4)

### 4.1 Undersampling + SMOTE

In [10]:
# ---------------------------
# 5. Setup Loss, Optimizer, and Training Parameters
# ---------------------------
# Since the training data is now balanced, we can use the default weights.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

num_epochs = 100
best_loss = float('inf')
best_model_path = "./Model_Weight/ResNet_best_under_smote_BIS.pt"

In [11]:
# ---------------------------
# 6. Training Loop with Checkpointing (Saving Best Model by Training Loss)
# ---------------------------
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # Save the model if training loss improved.
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), best_model_path)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [5/100], Loss: 0.6858
Epoch [10/100], Loss: 0.5663
Epoch [15/100], Loss: 0.4993
Epoch [20/100], Loss: 0.4563
Epoch [25/100], Loss: 0.4214
Epoch [30/100], Loss: 0.3944
Epoch [35/100], Loss: 0.3750
Epoch [40/100], Loss: 0.3618
Epoch [45/100], Loss: 0.3521
Epoch [50/100], Loss: 0.3442
Epoch [55/100], Loss: 0.3380
Epoch [60/100], Loss: 0.3329
Epoch [65/100], Loss: 0.3287
Epoch [70/100], Loss: 0.3252
Epoch [75/100], Loss: 0.3221
Epoch [80/100], Loss: 0.3193
Epoch [85/100], Loss: 0.3169
Epoch [90/100], Loss: 0.3147
Epoch [95/100], Loss: 0.3127
Epoch [100/100], Loss: 0.3109


In [12]:
# ---------------------------
# 7. Evaluation on the Test Set (with the Original Imbalanced Distribution)
# ---------------------------
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    
    y_pred = predicted.cpu().numpy()
    y_true = y_test.cpu().numpy()
    
    print("\nEvaluation on SetA (Test Data):")
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Precision-score:", precision_score(y_true, y_pred))
    print("Recall-score:", recall_score(y_true, y_pred))


Evaluation on SetA (Test Data):
              precision    recall  f1-score   support

           0       0.99      0.86      0.92    400000
           1       0.30      0.85      0.44     27655

    accuracy                           0.86    427655
   macro avg       0.64      0.86      0.68    427655
weighted avg       0.94      0.86      0.89    427655

[[343735  56265]
 [  4074  23581]]
F1-score: 0.43871219802606487
Precision-score: 0.29533101219848207
Recall-score: 0.8526848671126379


### 4.2 Normal SMOTE

In [9]:
# ---------------------------
# 5. Setup Loss, Optimizer, and Training Parameters
# ---------------------------
# Since the training data is now balanced, we can use the default weights.
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 200
best_loss = float('inf')
best_model_path = "./Model_Weight/ResNet_best_smote_BIS.pt"

In [10]:
# ---------------------------
# 6. Training Loop with Checkpointing (Saving Best Model by Training Loss)
# ---------------------------
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    # Save the model if training loss improved.
    if loss.item() < best_loss:
        best_loss = loss.item()
        torch.save(model.state_dict(), best_model_path)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [5/200], Loss: 0.7433
Epoch [10/200], Loss: 0.5814
Epoch [15/200], Loss: 0.5119
Epoch [20/200], Loss: 0.4620
Epoch [25/200], Loss: 0.4215
Epoch [30/200], Loss: 0.3915
Epoch [35/200], Loss: 0.3728
Epoch [40/200], Loss: 0.3599
Epoch [45/200], Loss: 0.3500
Epoch [50/200], Loss: 0.3413
Epoch [55/200], Loss: 0.3348
Epoch [60/200], Loss: 0.3296
Epoch [65/200], Loss: 0.3253
Epoch [70/200], Loss: 0.3217
Epoch [75/200], Loss: 0.3186
Epoch [80/200], Loss: 0.3159
Epoch [85/200], Loss: 0.3134
Epoch [90/200], Loss: 0.3112
Epoch [95/200], Loss: 0.3092
Epoch [100/200], Loss: 0.3073
Epoch [105/200], Loss: 0.3055
Epoch [110/200], Loss: 0.3039
Epoch [115/200], Loss: 0.3023
Epoch [120/200], Loss: 0.3008
Epoch [125/200], Loss: 0.2994
Epoch [130/200], Loss: 0.2981
Epoch [135/200], Loss: 0.2969
Epoch [140/200], Loss: 0.2957
Epoch [145/200], Loss: 0.2945
Epoch [150/200], Loss: 0.2934
Epoch [155/200], Loss: 0.2924
Epoch [160/200], Loss: 0.2914
Epoch [165/200], Loss: 0.2905
Epoch [170/200], Loss: 0.2896


In [11]:
# ---------------------------
# 7. Evaluation on the Test Set (with the Original Imbalanced Distribution)
# ---------------------------
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)
    
    y_pred = predicted.cpu().numpy()
    y_true = y_test.cpu().numpy()
    
    print("\nEvaluation on SetA (Test Data):")
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Precision-score:", precision_score(y_true, y_pred))
    print("Recall-score:", recall_score(y_true, y_pred))


Evaluation on SetA (Test Data):
              precision    recall  f1-score   support

           0       0.99      0.87      0.92    200000
           1       0.31      0.87      0.46     13828

    accuracy                           0.87    213828
   macro avg       0.65      0.87      0.69    213828
weighted avg       0.95      0.87      0.89    213828

[[173164  26836]
 [  1778  12050]]
F1-score: 0.45718404977804755
Precision-score: 0.3098801625263591
Recall-score: 0.8714203066242406


# 5. Evaluate on Unseen Data (Without Fine-tuning)

In [ ]:
best_model_path = "./Model_Final/CNN_best_model_smote_final_BIS.pt"

print(best_model_path)
loaded_model = FraudResNet(5, 128, 2, 5).to(device)
loaded_model.load_state_dict(torch.load(best_model_path))
loaded_model.eval()

In [ ]:
npy_files_unseen_1 = glob.glob("./BIS_data/client_part7_1.npy")

# Load each npy file into a list
client_data_list_unseen_1 = [np.load(file, allow_pickle=True) for file in npy_files_unseen_1]

# Combine all client data into one numpy array
combined_data_unseen_1 = np.vstack(client_data_list_unseen_1)

col_names = [f'pca_{i}' for i in range(5)] + ['laundering_schema_type', 'laundering_schema_id']

# Convert the combined array into a DataFrame
new_df = pd.DataFrame(combined_data_unseen_1, columns=col_names)
new_df = new_df.drop('laundering_schema_id', axis=1)
new_df = new_df.sample(frac=1, random_state=42).reset_index(drop=True)
new_df['laundering_schema_type'] = new_df['laundering_schema_type'].notna().astype(int)
new_df = new_df.sample(frac=1)

In [ ]:
# Without Fine-tuning
# ---------------------------
# 8. Load the Best Model into a New Instance and Test on Unseen Data (e.g., SetB)
# ---------------------------
# Create a new model instance with the same architecture.
# loaded_model = FraudMLP(input_dim, hidden_dim, output_dim)



if 'laundering_schema_type' in new_df.columns:
    X_new = new_df.drop('laundering_schema_type', axis=1).values
    y_new = new_df['laundering_schema_type'].values
else:
    X_new = new_df.values
    y_new = None

X_new = X_new.astype(np.float32)
X_new = torch.tensor(X_new, dtype=torch.float).to(device)
if y_new is not None:
    y_new = torch.tensor(y_new, dtype=torch.long).to(device)

with torch.no_grad():
    new_outputs = loaded_model(X_new)
    _, new_predicted = torch.max(new_outputs, 1)
    new_predictions = new_predicted.cpu().numpy()

if y_new is not None:
    # Convert y_new tensor to numpy array for metric calculations.
    y_new_np = y_new.cpu().numpy()
    print("\nEvaluation on Unseen Data (Set7_part1):")
    print(classification_report(y_new_np, new_predictions))
    print(confusion_matrix(y_new_np, new_predictions))
    print("F1-score:", f1_score(y_new_np, new_predictions))
    print("Precision-score:", precision_score(y_new_np, new_predictions))
    print("Recall-score:", recall_score(y_new_np, new_predictions))
else:
    print("\nPredictions on Unseen Data (Set7_part1):")
    print(new_predictions)

# 6. Evaluate on Unseen Data (With Fine-tuning)

In [ ]:
new_df = pd.read_csv("Fraud_dataset/Creditcard/GlobalUnseen.csv")
best_model_path = "./Model_Final/best_model_smote_final_all.pt"

# Separate features and labels.
X_new = new_df.drop('Class', axis=1).values
y_new = new_df['Class'].values

# Split the unseen data into a fine-tuning training set and a test set.
X_new_train, X_new_test, y_new_train, y_new_test = train_test_split(
    X_new, y_new, test_size=0.5, random_state=42, stratify=y_new
)

# Convert to PyTorch tensors.
X_new_train = torch.tensor(X_new_train, dtype=torch.float)
y_new_train = torch.tensor(y_new_train, dtype=torch.long)
X_new_test  = torch.tensor(X_new_test, dtype=torch.float)
y_new_test  = torch.tensor(y_new_test, dtype=torch.long)

# ---------------------------
# 2. Load the Pre-Trained Model from SetA
# ---------------------------
# Create a new model instance with the same architecture.
class FraudMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FraudMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.BatchNorm1d(hidden_dim),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        return self.model(x)

input_dim = X_new_train.shape[1]
hidden_dim = 64
output_dim = 2  # Two neurons for two classes

model = FraudMLP(input_dim, hidden_dim, output_dim)

loaded_model = FraudMLP(input_dim, hidden_dim, output_dim)
loaded_model.load_state_dict(torch.load(best_model_path))

# It's often useful to set the model in train mode during fine-tuning.
loaded_model.train()

# ---------------------------
# 3. Fine-Tuning on Unseen Data (SetB)
# ---------------------------
# Use a lower learning rate for fine-tuning.
finetune_optimizer = optim.Adam(loaded_model.parameters(), lr=1e-3)
finetune_criterion = nn.CrossEntropyLoss()

finetune_epochs = 300  # Adjust as needed

print("\n--- Fine-Tuning on Unseen Data (SetB) ---")
for epoch in range(finetune_epochs):
    finetune_optimizer.zero_grad()
    outputs = loaded_model(X_new_train)
    loss = finetune_criterion(outputs, y_new_train)
    loss.backward()
    finetune_optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f"Fine-Tuning Epoch [{epoch + 1}/{finetune_epochs}], Loss: {loss.item():.4f}")


In [ ]:
# ---------------------------
# 4. Evaluate the Fine-Tuned Model on SetB Test Data
# ---------------------------
loaded_model.eval()
with torch.no_grad():
    test_outputs = loaded_model(X_new_test)
    _, predicted = torch.max(test_outputs, 1)
    y_pred = predicted.numpy()
    y_true = y_new_test.numpy()
    
    print("\nEvaluation on Fine-Tuned Unseen Data (SetB Test):")
    print(classification_report(y_true, y_pred))
    print(confusion_matrix(y_true, y_pred))
    print("F1-score:", f1_score(y_true, y_pred))
    print("Precision-score:", precision_score(y_true, y_pred))
    print("Recall-score:", recall_score(y_true, y_pred))